In [ ]:
import os
import pandas as pd
import json
import numpy as np
import boto3
import cv2
from collections import defaultdict
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 500)

<h1> Define s3 credentials </h1>

In [ ]:
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")



<h1> Load and prepare fish detection dataset </h1>

In [ ]:
df = pd.read_csv('fish_detections.csv')
df.index = pd.to_datetime(df.created_at)


In [ ]:
df['left_image_width'] = df.left_crop_metadata.apply(lambda x: json.loads(x)['width'])
df['left_image_height'] = df.left_crop_metadata.apply(lambda x: json.loads(x)['height'])
df['left_x_coord'] = df.left_crop_metadata.apply(lambda x: json.loads(x)['x_coord'])
df['left_y_coord'] = df.left_crop_metadata.apply(lambda x: json.loads(x)['y_coord'])
df['right_image_width'] = df.right_crop_metadata.apply(lambda x: json.loads(x)['width'])
df['right_image_height'] = df.right_crop_metadata.apply(lambda x: json.loads(x)['height'])
df['right_x_coord'] = df.right_crop_metadata.apply(lambda x: json.loads(x)['x_coord'])
df['right_y_coord'] = df.right_crop_metadata.apply(lambda x: json.loads(x)['y_coord'])
df['left_mean_luminance'] = df.left_crop_metadata.apply(lambda x: json.loads(x)['mean_luminance'])
df['right_mean_luminance'] = df.right_crop_metadata.apply(lambda x: json.loads(x)['mean_luminance'])


In [ ]:
mask_L = (df.left_image_width + df.left_x_coord < 3900) & (df.left_image_height + df.left_y_coord < 2900) & (df.left_x_coord > 100) & (df.left_y_coord > 100)
mask_R = (df.right_image_width + df.right_x_coord < 3900) & (df.right_image_height + df.right_y_coord < 2900) & (df.right_x_coord > 100) & (df.right_y_coord > 100)
mask_orientation = (df.left_image_width / df.left_image_height) > 1.5
mask_luminance = df.left_mean_luminance > np.percentile(df.left_mean_luminance, 0)
mask = mask_L & mask_R & mask_orientation & mask_luminance
df[mask].sort_values('left_mean_luminance', ascending=False)



<h1> Iterate over fish detections selected from mask, rectify them, and upload them to s3 </h1>

In [ ]:
df.resample('D', how=lambda x: len(x))

In [ ]:
def load_params(params_file):
    params = json.load(open(params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)
    
    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps

def rectify_crop(crop_image_f, side, crop_metadata, left_maps, right_maps):
    maps = left_maps if side == 'left' else right_maps
    crop = cv2.imread(crop_image_f)
    new_image = np.zeros([3000, 4096, 3]).astype('uint8')
    lower_left = (crop_metadata['y_coord'] + crop_metadata['height'], crop_metadata['x_coord'])
    upper_right = (crop_metadata['y_coord'], crop_metadata['x_coord'] + crop_metadata['width'])
    new_image[upper_right[0]:lower_left[0], lower_left[1]:upper_right[1], :] = np.array(crop)
    remap = cv2.remap(new_image, maps[0], maps[1], cv2.INTER_LANCZOS4)
    cv2.imwrite('{}.jpg'.format(side), remap)
    nonzero_indices = np.where(remap > 0)
    y_min, y_max = nonzero_indices[0].min(), nonzero_indices[0].max() 
    x_min, x_max = nonzero_indices[1].min(), nonzero_indices[1].max()
    lower_left = (y_max, x_min)
    upper_right = (y_min, x_max)
    rectified_crop = remap[upper_right[0]:lower_left[0], lower_left[1]:upper_right[1], :].copy()
    
    crop_image_fname = os.path.basename(crop_image_f)
    outfile_fname = '{}_rectified.jpg'.format(crop_image_fname.replace('.jpg', ''))
    outfile_f = os.path.join(os.path.dirname(crop_image_f), outfile_fname)
    cv2.imwrite(outfile_f, rectified_crop)
    return outfile_f

In [ ]:
stereo_parameters_f = './stereo_parameters_old_blom_pen_1_enclosure.json'
left_maps, right_maps = load_params(stereo_parameters_f)

In [ ]:
tdf = df[mask].sort_values('left_mean_luminance', ascending=False)

s3_bucket = 'aquabyte-crops'
s3_prefix = 'https://s3-eu-west-1.amazonaws.com'

urls_by_date = defaultdict(list)

i = 0
for idx, row in tdf.iterrows():
    
    date = row.captured_at[:10]
    
    left_crop_url, right_crop_url = row.left_image_url, row.right_image_url
    left_crop_key = left_crop_url[left_crop_url.index('environment'):]
    right_crop_key = right_crop_url[right_crop_url.index('environment'):]
    left_crop_f = os.path.basename(left_crop_url)
    right_crop_f = os.path.basename(right_crop_url)
    left_crop_metadata = json.loads(row.left_crop_metadata)
    right_crop_metadata = json.loads(row.right_crop_metadata)
    
    s3_client.download_file(s3_bucket, left_crop_key, left_crop_f)
    s3_client.download_file(s3_bucket, right_crop_key, right_crop_f)
    rectified_left_crop_f = rectify_crop(left_crop_f, 'left', left_crop_metadata, left_maps, right_maps)
    rectified_right_crop_f = rectify_crop(right_crop_f, 'right', right_crop_metadata, left_maps, right_maps)
    left_key = os.path.join('rectified_crops', date, rectified_left_crop_f)
    right_key = os.path.join('rectified_crops', date, rectified_right_crop_f)
    s3_client.upload_file(rectified_left_crop_f, 'aquabyte-images-adhoc', left_key)
    s3_client.upload_file(rectified_right_crop_f, 'aquabyte-images-adhoc', right_key)
    left_url = os.path.join(s3_prefix, 'aquabyte-images-adhoc', left_key)
    right_url = os.path.join(s3_prefix, 'aquabyte-images-adhoc', right_key)
    urls_by_date[date].append(left_url)
    urls_by_date[date].append(right_url)
    os.remove(left_crop_f)
    os.remove(rectified_left_crop_f)
    os.remove(right_crop_f)
    os.remove(rectified_right_crop_f)
    if i % 10 == 0:
        print(i)
    i += 1
    
    
    
    
    

In [ ]:
urls_by_date['2019-04-23']

In [ ]:
left_crop_f

In [ ]:
left_crop_metadata